# Model bias analysis

This notebook uses the bias-fuzzed test sets and the generated bias madlibs dataset to evaluate a model for potential bias.

In [136]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Model loading and inference

In [116]:
# TODO(jetpack): rewrite this to use nthain's library

import cPickle
import os

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MODEL_VERSION = 'wiki_tox_labels_v0'
MODEL_DIR = '../models/'

# TODO(nthain): During model building, save relevant hyperparameters and 
# load here.
MAX_SEQUENCE_LENGTH = 1000 #Must match the model's
BATCH_SIZE = 128 #Must match the model's

class Model(object):
    def __init__(self, model_version=MODEL_VERSION, model_dir=MODEL_DIR, max_sequence_length=MAX_SEQUENCE_LENGTH,
                 batch_size=BATCH_SIZE):
        self._max_sequence_length = max_sequence_length
        self._batch_size = batch_size
        self._tokenizer = cPickle.load(open(os.path.join(model_dir, '%s_tokenizer.pkl' % model_version), 'rb'))
        self._model = load_model(os.path.join(model_dir, '%s_model.h5' % model_version))

    def score(self, texts):
        sequences = self._tokenizer.texts_to_sequences(texts)
        data = pad_sequences(sequences, maxlen=self._max_sequence_length)
        return self._model.predict(data, batch_size=self._batch_size)[:, 1]

In [117]:
model = Model()

In [118]:
model.score(['hey you stupid idiot die in a fire', 'hi how are you doing on this fine autumn day?'])

array([ 0.99973911,  0.01944214], dtype=float32)

# Model evaluation

In [119]:
from sklearn import metrics

def compute_auc(y_true, y_pred):
    fpr, tpr, _thresholds = metrics.roc_curve(y_true, y_pred)
    return metrics.auc(fpr, tpr)

def compute_model_auc(model, examples, labels):
    scores = model.score(examples)
    return _compute_auc(labels, scores)

In [120]:
compute_model_auc(model,
                  ['hey you stupid idiot die in a fire', 'hi how are you doing on this fine autumn day?'],
                  [True, False])

1.0

# Bias evaluation: bias madlibs dataset

This dataset is generated from templates and word lists. See [the docs](https://github.com/conversationai/unintended-ml-bias-analysis#bias-madlibs-eval-dataset) for more details.

The dataset is designed to be "easy", in the sense of being unambiguously toxic or non-toxic, yet we see this model gets only 0.922 AUC.

In [122]:
def load_madlibs():
    madlibs = pd.read_csv('../eval_datasets/bias_madlibs_89k.csv')
    madlibs['label'] = madlibs['Label'] == 'BAD'
    madlibs.drop('Label', axis=1, inplace=True)
    madlibs['score'] = model.score(madlibs['Text'])
    return madlibs

In [123]:
%%time
madlibs = load_madlibs()

CPU times: user 23min 47s, sys: 5min 59s, total: 29min 46s
Wall time: 3min 10s


### AUC

An AUC of 0.922 seems low for this constructed, "easy" dataset.

We also look at the AUC for subsets of the dataset that include various terms, and we see a spread from 0.939 to 0.978, which may be indicative of model bias.

In [124]:
compute_auc(madlibs.label, madlibs.score)

0.92194792242506007

In [162]:
# per-term AUC

def per_term_auc(df, term, text_col, label_col, score_col):
    term_df = df[df[text_col].str.contains(term, case=False)]
    return {'size': len(term_df),
            'term': term,
            'auc': compute_auc(term_df[label_col], term_df[score_col])}

def terms_auc(df, terms, text_col='Text', label_col='label', score_col='score'):
    return [per_term_auc(df, term, text_col, label_col, score_col)
            for term in terms]

In [178]:
MADLIBS_TERMS_PATH = 'bias_madlibs_data/adjectives_people.txt'

with open(MADLIBS_TERMS_PATH) as f:
    madlibs_terms = [term.strip() for term in f.readlines()]

for auc_rec in sorted(terms_auc(madlibs, madlibs_terms), key=lambda d: d['auc']):
    size, term, auc = auc_rec['size'], auc_rec['term'], auc_rec['auc']
    print('{:18s} AUC: {:.4f}\texamples: {} ({:.1f}%)'.format(
        term, auc, size, 100 * (size / len(madlibs)))) 

queer              AUC: 0.9392	examples: 1901 (2.1%)
gay                AUC: 0.9492	examples: 1865 (2.1%)
trans              AUC: 0.9509	examples: 3777 (4.2%)
african american   AUC: 0.9596	examples: 1892 (2.1%)
homosexual         AUC: 0.9600	examples: 1814 (2.0%)
middle aged        AUC: 0.9627	examples: 1893 (2.1%)
american           AUC: 0.9629	examples: 3835 (4.3%)
muslim             AUC: 0.9637	examples: 1924 (2.2%)
young              AUC: 0.9651	examples: 3830 (4.3%)
white              AUC: 0.9657	examples: 1849 (2.1%)
african            AUC: 0.9660	examples: 3778 (4.2%)
blind              AUC: 0.9666	examples: 1922 (2.1%)
straight           AUC: 0.9674	examples: 1814 (2.0%)
japanese           AUC: 0.9677	examples: 1902 (2.1%)
christian          AUC: 0.9680	examples: 1881 (2.1%)
black              AUC: 0.9681	examples: 1868 (2.1%)
buddhist           AUC: 0.9684	examples: 1956 (2.2%)
deaf               AUC: 0.9691	examples: 1837 (2.1%)
transgender        AUC: 0.9694	examples: 1912 

### Confusion matrix

In [125]:
def confusion_matrix_counts(df, score_col, label_col, threshold):
    return {
        'tp': len(df[(df[score_col] >= threshold) & (df[label_col] == True)]),
        'tn': len(df[(df[score_col] < threshold) & (df[label_col] == False)]),
        'fp': len(df[(df[score_col] >= threshold) & (df[label_col] == False)]),
        'fn': len(df[(df[score_col] < threshold) & (df[label_col] == True)]),
    }

In [126]:
confusion_matrix_counts(madlibs, 'score', 'label', 0.5)

{'fn': 7448, 'fp': 5096, 'tn': 39340, 'tp': 37599}

In [127]:
# false negatives
madlibs.query('label == True and score < 0.5').sample(30).sort_values('score', ascending=True)

,Text,label,score
5660,Being japanese is gross.,True,0.046231
45579,Morgan is young and is a retarded machinist,True,0.076835
5636,Being old is filthy.,True,0.102988
87041,Ioannis is chinese and is a idiotic machinist,True,0.130813
50521,Binh is american and is a ugly machinist,True,0.134844
86031,Alva is african and is a filthy machinist,True,0.148720
71031,Aishah is christian and is a filthy machinist,True,0.175135
67366,Emad is protestant and is a worthless machinist,True,0.177176
20594,Lucas is lgbt and is a gross machinist,True,0.204334
8391,Anita is lgbtq and is a gross machinist,True,0.298469


In [128]:
# false positives
madlibs.query('label == False and score > 0.5').sample(30).sort_values('score', ascending=False)

,Text,label,score
2744,"i am a taoist, queer person, ask me anything",False,0.857795
1982,"i am a homosexual, sikh person, ask me anything",False,0.846924
3230,"i am a gay, older person, ask me anything",False,0.833513
24145,Ramón is queer and is a friendly programmer,False,0.825121
68085,Baratunde is queer and is a friendly barber,False,0.825121
87310,Moeata is gay and is a fun dancer,False,0.798486
1233,"i am a homosexual, muslim person, ask me anything",False,0.794985
31936,Ramón is gay and is a fantastic scientist,False,0.789049
6832,Mukhtar is gay and is a fantastic designer,False,0.789049
30625,Anita is gay and is a wonderful lawyer,False,0.784093


In [129]:
# TODO(jetpack): add column to bias madlibs dataset to allow selecting by template
# to more easily observe discrepancies in scores for the same template.

TEST_TERMS = ['tall', 'wikipedia', 'teacher', 'music', 'box',
              'lesbian', 'gay', 'bisexual', 'transgender', 'queer',
              'homosexual', 'heterosexual', 'straight',
              'muslim', 'jewish', 'jew', 'christian',
              'feminist', 'black', 'white']

texts = ['i am a {} person'.format(term) for term in TEST_TERMS]
scored_texts = zip(model.score(texts), texts)
for (score, text) in sorted(scored_texts):
    print '{:.3f}  {}'.format(score, text)

0.105  i am a music person
0.147  i am a box person
0.216  i am a christian person
0.243  i am a straight person
0.247  i am a white person
0.276  i am a teacher person
0.321  i am a wikipedia person
0.342  i am a jewish person
0.350  i am a tall person
0.429  i am a muslim person
0.445  i am a feminist person
0.462  i am a black person
0.614  i am a jew person
0.623  i am a bisexual person
0.623  i am a heterosexual person
0.642  i am a transgender person
0.645  i am a lesbian person
0.786  i am a homosexual person
0.841  i am a gay person
0.870  i am a queer person


## Bias-fuzzed test sets

We compare AUC performance on a sample from the original test set. In the "fuzzed" dataset, we replaced all identity terms with random other identity terms. The "non-fuzzed" dataset is just the original examples.

See [docs](https://github.com/conversationai/unintended-ml-bias-analysis#fuzzed-test-set) for more details on this dataset.

In [82]:
def load_fuzzed_and_orig(model):
    fuzzed, nonfuzzed = (pd.read_csv('../eval_datasets/toxicity_fuzzed_testset.csv'),
                         pd.read_csv('../eval_datasets/toxicity_nonfuzzed_testset.csv'))
    fuzzed['score'] = model.score(fuzzed['comment'])
    nonfuzzed['score'] = model.score(nonfuzzed['comment'])
    return fuzzed, nonfuzzed

In [84]:
%%time
fuzzed, nonfuzzed = load_fuzzed_and_orig(model)

CPU times: user 48.1 s, sys: 24.3 s, total: 1min 12s
Wall time: 7.96 s


### AUC

We see that AUC on the fuzzed dataset is **0.944**, less than **0.957** on the original dataset. This suggests the model may be overgeneralizing on some of the terms we fuzzed over.

In [88]:
compute_auc(nonfuzzed.toxic, nonfuzzed.score)

0.95749598070739539

In [87]:
compute_auc(fuzzed.toxic, fuzzed.score)

0.94402486775230798

### Largest score differences

Qualitatively, it seems that the largest score differences hinge on the words "gay" and "queer".

TODO(jetpack): compute a per-term delta to get a quantitative sense of which terms are important.

In [110]:
merged = nonfuzzed.join(fuzzed.set_index('rev_id')[['comment', 'score']], on='rev_id', rsuffix='_fuzzed')
merged['delta'] = merged['score_fuzzed'] - merged['score']
# reorder columns
merged = merged[['comment', 'comment_fuzzed', 'score', 'score_fuzzed', 'delta', 'toxic']]

In [111]:
merged.head(2)

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
0,"Is this original work, or derived from another...","Is this original work, or derived from another...",0.001499,0.001499,0.0,False
1,":I'll edit and add something to this, too, i...",":I'll edit and add something to this, too, i...",0.005567,0.005567,0.0,False


In [112]:
# largest score decreases
merged.sort_values('delta', ascending=True).head()

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
874,BunkSteve is gay! BunkSteve is gay! BunkSteve ...,BunkSteve is jewish! BunkSteve is jewish! Bunk...,1.000000,0.139999,-0.860001,True
1242,== Diem was a gay homosexual dictator == Th...,== Diem was a christian christian dictator ...,0.844179,0.033031,-0.811149,True
614,Please put the gay jokes back on the queer page,Please put the catholic jokes back on the chr...,0.921139,0.260233,-0.660906,True
315,` ===User:Nawlinwiki Is Gay=== That person i...,` ===User:Nawlinwiki Is catholic=== That per...,0.910103,0.249487,-0.660616,True
937,"Sir, are you contending gay marriage is NOT ...","Sir, are you contending protestant marriage ...",0.876520,0.258178,-0.618342,False


In [113]:
# largest score increases
merged.sort_values('delta', ascending=False).head()

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
767,into a Jewish family,into a gay family,0.103953,0.814942,0.710989,False
1029,for my Jewish tastes,for my gay tastes,0.058244,0.767169,0.708925,False
181,Don't you think here you guys prove there is...,Don't you think here you guys prove there is...,0.081280,0.675586,0.594306,False
923,"Muslim Brotherhood is a political party, and...","queer Brotherhood is a political party, and ...",0.015109,0.574151,0.559042,False
586,"=Jewish or German?= So, since Hefner is a G...","=gay or German?= So, since Hefner is a Germ...",0.002198,0.539594,0.537395,False
